In [1]:
import brunoflow as bf
from brunoflow.ad.utils import check_node_equals_tensor
from jax import numpy as jnp, random

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# hyperparameters
random_key_val = 42
num_embeddings = 5
embedding_dim = 3
padding_idx = 1
random_key = random.PRNGKey(random_key_val)


2022-12-22 16:01:18.264866: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [3]:
# create and inspect a torch embedding
import torch
emb = torch.nn.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
emb.weight

Parameter containing:
tensor([[ 0.0813,  1.1262,  0.3836],
        [ 0.0000,  0.0000,  0.0000],
        [-1.1691,  0.5396,  0.3151],
        [-0.1995, -0.8717,  1.5982],
        [-0.1942, -0.6943, -0.2199]], requires_grad=True)

In [4]:
# Create and inspect a bf Embedding
emb_bf = bf.net.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
bf.matmul(emb_bf([2]), jnp.ones(shape=(3, 4))).backprop()
print(emb_bf.weight.grad)

[[0. 0. 0.]
 [0. 0. 0.]
 [4. 4. 4.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [5]:
# Inspect grad for a non-pad token - note that the grad is the same for bf and torch!
out = torch.matmul(emb(torch.tensor([2])), torch.ones(size=(3, 4), requires_grad=True))
out.backward(gradient=torch.ones_like(out))
assert(jnp.array_equal(emb_bf.weight.grad, emb.weight.grad))
emb.weight.grad


/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/torch/autograd/__init__.py:173: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664405705473/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [4., 4., 4.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [6]:
# Inspect grad for a pad token for torch - note that the grad DID NOT change from before in the 1st (pad token index) row
out = torch.matmul(emb(torch.tensor([1])), torch.ones(size=(3, 4), requires_grad=True))
out.backward(gradient=torch.ones_like(out))
emb.weight.grad

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [4., 4., 4.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [7]:
# Inspect grad for a pad token for bf, note that the grad DID NOT change from before in the pad token index. Same behavior as torch
bf.matmul(emb_bf([1]), jnp.ones(shape=(3, 4))).backprop()
print(emb_bf.weight.grad)
assert(jnp.array_equal(emb_bf.weight.grad, emb.weight.grad.numpy()))

[[0. 0. 0.]
 [0. 0. 0.]
 [4. 4. 4.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [8]:
# Check that bf and torch embedding contain the same modules 
print("bf modules:", [i for i in emb_bf.modules()])
print("torch modules:", [i for i in emb.modules()])

bf modules: [Embedding(5, 3, padding_idx=1)]
torch modules: [Embedding(5, 3, padding_idx=1)]


In [9]:
# Check that we can access parameters of both bf and torch embeddings
print([i for i in emb_bf.parameters()])
print([(i, i.grad) for i in emb.parameters()])

[node(name: emb weights (5, 3), val: [[-0.716899   -0.20865498 -2.5713923 ]
 [ 0.          0.          0.        ]
 [-0.8396519   0.3010434   0.1421263 ]
 [-1.7631724  -1.6755073   0.31390068]
 [ 0.5912831   0.5325395  -0.9133108 ]], grad: [[0. 0. 0.]
 [0. 0. 0.]
 [4. 4. 4.]
 [0. 0. 0.]
 [0. 0. 0.]])]
[(Parameter containing:
tensor([[ 0.0813,  1.1262,  0.3836],
        [ 0.0000,  0.0000,  0.0000],
        [-1.1691,  0.5396,  0.3151],
        [-0.1995, -0.8717,  1.5982],
        [-0.1942, -0.6943, -0.2199]], requires_grad=True), tensor([[0., 0., 0.],
        [0., 0., 0.],
        [4., 4., 4.],
        [0., 0., 0.],
        [0., 0., 0.]]))]


In [10]:
# Check that buffers are the same between bf and torch embeddings
print([i for i in emb_bf.buffers()])
print([i for i in emb.buffers()])

[]
[]


### Saving a torch embedding state dict and reloading it as bf

In [11]:
save_torch_path = "emb.pt"
torch.save(emb.state_dict(), save_torch_path)

In [12]:
emb_loaded = torch.nn.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
emb_loaded.load_state_dict(torch.load(save_torch_path))


<All keys matched successfully>

In [13]:
# check that bf embedding when loading from a torch state dict ends up with same values
emb_bf_loaded = bf.net.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
emb_bf_loaded.load_state_dict(torch.load(save_torch_path))
print(emb_bf_loaded.weight)
print(emb_loaded.weight)
assert(check_node_equals_tensor(emb_bf_loaded.weight, emb_loaded.weight))

node(name: emb weights (5, 3), val: [[ 0.08126468  1.1262476   0.38362706]
 [ 0.          0.          0.        ]
 [-1.1691363   0.5395742   0.3151424 ]
 [-0.19945197 -0.87169373  1.5982424 ]
 [-0.19424585 -0.69431293 -0.21993046]], grad: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]])
Parameter containing:
tensor([[ 0.0813,  1.1262,  0.3836],
        [ 0.0000,  0.0000,  0.0000],
        [-1.1691,  0.5396,  0.3151],
        [-0.1995, -0.8717,  1.5982],
        [-0.1942, -0.6943, -0.2199]], requires_grad=True)


In [14]:
# Check forward and backward passes to make sure the outputs match
inds = [1,2,3]
embs_bf_out = emb_bf_loaded(jnp.array(inds))
embs_out = emb_loaded(torch.tensor(inds))
print("bf output of emb:", embs_bf_out)
print("torch output of emb:", embs_out)
assert(check_node_equals_tensor(embs_bf_out, embs_out))

embs_bf_out.backprop()
embs_out.backward(gradient=torch.ones_like(embs_out))
# print(embs_bf_out.grad, embs_out.grad)
assert(jnp.array_equal(emb_bf_loaded.weight.grad, emb_loaded.weight.grad.numpy()))
print(emb_bf_loaded.weight.grad, emb_loaded.weight.grad)

bf output of emb: node(name: get_embedding, val: [[ 0.          0.          0.        ]
 [-1.1691363   0.5395742   0.3151424 ]
 [-0.19945197 -0.87169373  1.5982424 ]], grad: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]])
torch output of emb: tensor([[ 0.0000,  0.0000,  0.0000],
        [-1.1691,  0.5396,  0.3151],
        [-0.1995, -0.8717,  1.5982]], grad_fn=<EmbeddingBackward0>)
[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 1.]
 [0. 0. 0.]] tensor([[0., 0., 0.],
        [0., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.]])


In [15]:
emb_bf_loaded = bf.net.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
emb_bf_loaded.load_state_dict(torch.load(save_torch_path))
out = bf.get_embedding(emb_bf_loaded.weight, [0, 1, 2, 3, 4], padding_idx=1)
out.backprop()
print(emb_bf_loaded.weight.grad)

[[1. 1. 1.]
 [0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
